In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from bs4 import BeautifulSoup
from Bio.SeqFeature import CompoundLocation

### web scraping (escolher condição e nº de genes)

In [2]:
#escolha do query
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
#     print (a)

exist = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(exist)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20) atenção que o reultado pode ser menor que o desejado: '))] #can be more. related to ncbi pagination
print(n_genes)

if n_genes == ['']:
    print ("Não encontramos resultado para a sua pesquisa")
# else:
#     print (n_genes)

escolha o que quer pesquisar: diabtes
escolha o nº de genes que quer obter (máximo 20) atenção que o reultado pode ser menor que o desejado: 10
['16847', '1493', '7422', '3569', '1029', '5468', '387082', '79068', '3630', '6934']


### Buscar Links dos genes:

In [66]:
# FINAL - necessário em pubmed (para script do rodrigo)
def url_get_id(i):
    url_list= [ ]
    for id in i:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format(id)
        url_list.append(url)
    return url_list
# url_get_id(Ids) 

### Buscar ids do ncbi: Ids

In [3]:
# FINAL - Ids   (NECESSÁRIO)
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
print(Ids)

C:\Users\Karyna\anaconda3\lib\site-packages\Bio\Entrez\__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


['Z17674.1', 'V00883.1', 'X67017.1', 'X61243.1']


### Buscar info para CDS: protein_id, location e product

In [4]:
#FINAL para protein_id - PROT_ID (NECESSÁRIO)
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
print (ID_PROT) # is used below in Uniprot

['CAA24252.1', 'CAA47407.1']


In [5]:
#FINAL para location, product e translation  - get_CDS_info(Ids) = [id_protein, location, product, translation] - NECESSÁRIO
def get_CDS_info(id_list):
    database = 'nucleotide'
    email= 'rodrigoce9@gmail.com'
    Entrez.email = email
    handle = Entrez.efetch(db=database, id=id_list, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    cds_location_list = []
    
    several_location = []
    record_types={}
    for i in id_list:
        handle2 = Entrez.efetch(db=database, id=i, rettype="gb")
        for record in SeqIO.parse(handle2, "genbank"):
            for feature in record.features:
                record_types[feature.type]=record_types.get(feature.type,0)+1
                cds_nonlocation, nonlocation, nonproduct, nontranslation = [], [], [], []
            if 'CDS' not in record_types:
                cds_nonlocation.append("N/A_CDS")
                nonlocation.append("N/A")
                nonproduct.append("N/A")
                nontranslation.append("N/A")
                cds_nonlocation.append(nonlocation)
                cds_nonlocation.append(nonproduct)
                cds_nonlocation.append(nontranslation)
                cds_location_list.append(cds_nonlocation)
#                 cds_location_list.append(nonlocation)
                
    handle2.close()
    for info in records:
        for i in info.features:
            list_product, list_translation = [],[]
            cds_location = []
            product, translation = '', ''
            if i.type == "CDS":
                i_d = str(i.qualifiers["protein_id"])
                product = str(i.qualifiers["product"])
                translation =  str(i.qualifiers["translation"])
                cds_location.append(i_d)
                if isinstance(i.location, CompoundLocation):
                    for sub_location in i.location.parts:
                        several_location.append("[{} : {}]".format(sub_location.start, sub_location.end))
                    cds_location.append(several_location)
                    cds_location.append(product)
                    cds_location.append(translation)
                else:
                    cds_location.append("[{} : {}]".format(i.location.start, i.location.end))
                    cds_location.append(product)
                    cds_location.append(translation)
                cds_location_list.append(cds_location)      
    handle2.close()                    
    return cds_location_list
print(get_CDS_info(Ids))

[['N/A_CDS', ['N/A'], ['N/A'], ['N/A']], ["['CAA24252.1']", ['[223 : 316]', '[440 : 662]', '[1479 : 1608]'], "['beta-3 globin']", "['MVHFTAEEKAAITSTWKLVDVEDAGAEALGRLLVVYPWTQRFFDSFGNLSSSSAIMGNPKVKAHGKKVLTAFGDAVKNVDDLKNTFAHLSELHCDRLHVDPENFKLLGNVLVIVLAKYFGKEFTPQVQSAWQKLVAGVATALAHKYH']"], ["['CAA47407.1']", '[246 : 921]', "['cytochrome c1 heme lyase']", "['MMSSDQQGKCPVDEETKKLWLREHGNEAHPGATAPGNQLECSANPQDNDKTPEYHTTVDLSQSREVSTIPRTNSDRNWIYPSEKQFYEAMMKKNWDPNSDDMKVVVPLHNSINERVWNYIKSWEDKQGGEACGGIKLTNFKGDSKKLTPRAWFRSRILHLAKPFDRHDWQIDRCGKTVDYVIDFYSTDLNDANSQQQPLIYLDVRPKLNSFEGFRLRFWKSLGF']"]]


In [7]:
# for i in get_CDS_info(Ids):
#     print(i)

['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]


In [61]:
#original para product
# for ids in Ids:
#     list_pro=[]
#     handle = Entrez.efetch(db=database, id=ids, rettype="gb") 
#     records = list(SeqIO.parse(handle,"gb"))
#     handle.close()
#     for info in records:
#         list_pro.append(info.id)
#         for i in info.features:
#             if i.type == "CDS":
#                 product= str(i.qualifiers["product"])
#                 print(product)
#                 list_pro.append(pro)

In [6]:
#FINAL result_dic = {Ids, ID_PROT} - NECESSÁRIO
result_dict = {}
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist=Ids
# idlist= Ids.remove('U49845')
for ids in idlist:
    list_pro=[]
    handle = Entrez.efetch(db=database, id=ids, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    for info in records:
        list_pro.append(info.id)
        for i in info.features:
            if i.type == "CDS":
                pro= str(i.qualifiers["protein_id"])
                list_pro.append(pro)
        if len(list_pro)==1: #if no protein_id was found
            result_dict[info.id] = "N/A_CDS"
        else:
            list_pro = [x.replace("['",'').replace("']",'') for x in list_pro]
            result_dict[info.id] = ', '.join(list_pro[1:])
print(result_dict)
#key: id_genebank; values: id_Uniprot

{'Z17674.1': 'N/A_CDS', 'V00883.1': 'CAA24252.1', 'X67017.1': 'CAA47407.1', 'X61243.1': 'N/A_CDS'}


### Buscar Links de proteinas:

In [7]:
#final, mas não correr (código necessário para o caso se querermos links não diretos para uniprot )
def url_get_id_p(dic):
    url_list_p = []
    for key, value in dic.items():
        if value != 'N/A_CDS':
            value_list = value.split(',')
            for val in value_list:
                val = val.strip()
                url_id_p = "https://www.uniprot.org/uniprotkb?query={}".format(val)
                url_list_p.append(url_id_p)
    return url_list_p
url_ids_protein=url_get_id_p(result_dict)
print(url_ids_protein) # but we don't need to save this links

['https://www.uniprot.org/uniprotkb?query=CAA24252.1', 'https://www.uniprot.org/uniprotkb?query=CAA47407.1']


### Buscar Links de proteinas na Uniprot:

In [8]:
#FINAL (usado em baixo) - NECESSÁRIO
def url_get_id_p(string):
#     print(string)
    url_list_p=[ ]
    url_id_p= "https://www.uniprot.org/uniprotkb/{}/entry".format(str(string))
    url_list_p.append(url_id_p)
    return ''.join(url_list_p)
print(url_get_id_p)

<function url_get_id_p at 0x00000187FD752790>


In [61]:
# for index, value in enumerate(get_list_uniprot(ID_PROT)):
#     print(get_list_uniprot(ID_PROT)[index])
#     print(get_list_uniprot(ID_PROT)[index][1])
    

### Buscar id proteina (uniprot):

In [10]:
#origianl ------------------
WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()
    return response

def get_field_for_id(ID_PROT, field):
    response = get_url("{}/uniprotkb/search?query={}&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)

def get_list_uniprot(ID_PROT):
    results = {}
    for i in ID_PROT:
        result = {}
        for field in fields:
            result[field] = get_field_for_id(i, field)
        results[i] = result

    uniprot_final_list=[]
    for key, value in results.items():
        uniprot_list=[]
        for x in value.values():
#             print(key)
            entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL)
            function = re.match( r'b\'Function \[CC\]\\n.{9} (.+?(?=\\n\'))', x, re.DOTALL )
            location_exist = re.search( r'b\'Subcellular location \[CC\]\\nSUBCELLULAR LOCATION: (.+?(?=\\n\'))', x, re.DOTALL )
            location_notexist = re.search( r'b\'Subcellular location \[CC\]\\n\\n\'', x, re.DOTALL )   
            sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
            if entry:
                ent=entry.group(1)
                uniprot_list.append(entry.group(1))
            if function:
                uniprot_list.append(function.group(1))
#                 print(function.group(0))
#             if not function:
#                 print('na')
            if location_exist:
                uniprot_list.append(location_exist.group(1))
            if location_notexist:
                uniprot_list.append("N/A")
            if sequence:
                uniprot_list.append(sequence.group(1))
                uniprot_list.append(len(sequence.group(1)))
                uniprot_list.append(url_get_id_p(ent))
                
        uniprot_final_list.append(uniprot_list)
        
    return uniprot_final_list
print(get_list_uniprot(ID_PROT)) #list of lists (ID_Uniprot, Function, Subcelular location, Protein seq, length_aa)

[['P02099', 'N/A', 'This protein functions as an embryonic globin, but the gene structure and chromosomal location resemble more closely the human gamma chain gene, which codes for a fetal globin.', 'MVHFTAEEKAAITSTWKLVDVEDAGAEALGRLLVVYPWTQRFFDSFGNLSSSSAIMGNPKVKAHGKKVLTAFGDAVKNVDDLKNTFAHLSELHCDRLHVDPENFKLLGNVLVIVLAKYFGKEFTPQVQSAWQKLVAGVATALAHKYH', 147, 'https://www.uniprot.org/uniprotkb/P02099/entry'], ['Q00873', 'Mitochondrion inner membrane {ECO:0000269|PubMed:1499554}. Note=Partially exposed to the intermembrane space. {ECO:0000305|PubMed:1499554}.', 'Lyase that catalyzes the covalent linking of the heme group to the cytochrome C1 apoprotein to produce the mature functional cytochrome. {ECO:0000269|PubMed:1499554}.', 'MMSSDQQGKCPVDEETKKLWLREHGNEAHPGATAPGNQLECSANPQDNDKTPEYHTTVDLSQSREVSTIPRTNSDRNWIYPSEKQFYEAMMKKNWDPNSDDMKVVVPLHNSINERVWNYIKSWEDKQGGEACGGIKLTNFKGDSKKLTPRAWFRSRILHLAKPFDRHDWQIDRCGKTVDYVIDFYSTDLNDANSQQQPLIYLDVRPKLNSFEGFRLRFWKSLGF', 224, 'https://www.uniprot.org/uniprotkb/Q0

In [14]:
#FINAL - NECESSÁRIO
WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()
    return response

def get_field_for_id(ID_PROT, field):
    response = get_url("{}/uniprotkb/search?query={}&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)

def get_list_uniprot(ID_PROT, result_dict):
    results = {}
    result = {}
    for first_key, first_value in result_dict.items(): 
        result = {}
        for field in fields:
            if first_value != 'N/A_CDS':
                result[field] = get_field_for_id(first_value, field)
            else:
                result[field] = 'N/A'
            results[first_key] = result   
    
    uniprot_final_list=[]
    for key, value in results.items():
        uniprot_list=[]
        for x in value.values():
#             print(key)
            entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL)
            function = re.match( r'b\'Function \[CC\]\\n.{9} (.+?(?=\\n\'))', x, re.DOTALL )
            location_exist = re.search( r'b\'Subcellular location \[CC\]\\nSUBCELLULAR LOCATION: (.+?(?=\\n\'))', x, re.DOTALL )
            location_notexist = re.search( r'b\'Subcellular location \[CC\]\\n\\n\'', x, re.DOTALL )   
            sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
            n_a = re.search(r'(N/A)', x, re.DOTALL)
            if entry:
                ent=entry.group(1)
                uniprot_list.append(entry.group(1))            
            if function:
                uniprot_list.append(function.group(1))
#                 print(function.group(0))
#             if not function:
#                 print('na')
            if location_exist:
                uniprot_list.append(location_exist.group(1))
            if location_notexist:
                uniprot_list.append("N/A")
            if sequence:
                uniprot_list.append(sequence.group(1))
                uniprot_list.append(len(sequence.group(1)))
                uniprot_list.append(url_get_id_p(ent))
            if n_a:
                uniprot_list.append(n_a.group(1))
                
        uniprot_final_list.append(uniprot_list)
        
    return uniprot_final_list
print(get_list_uniprot(ID_PROT,result_dict)) #list of lists (ID_Uniprot, Function, Subcelular location, Protein seq, length_aa)

[['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A'], ['P02099', 'N/A', 'This protein functions as an embryonic globin, but the gene structure and chromosomal location resemble more closely the human gamma chain gene, which codes for a fetal globin.', 'MVHFTAEEKAAITSTWKLVDVEDAGAEALGRLLVVYPWTQRFFDSFGNLSSSSAIMGNPKVKAHGKKVLTAFGDAVKNVDDLKNTFAHLSELHCDRLHVDPENFKLLGNVLVIVLAKYFGKEFTPQVQSAWQKLVAGVATALAHKYH', 147, 'https://www.uniprot.org/uniprotkb/P02099/entry'], ['Q00873', 'Mitochondrion inner membrane {ECO:0000269|PubMed:1499554}. Note=Partially exposed to the intermembrane space. {ECO:0000305|PubMed:1499554}.', 'Lyase that catalyzes the covalent linking of the heme group to the cytochrome C1 apoprotein to produce the mature functional cytochrome. {ECO:0000269|PubMed:1499554}.', 'MMSSDQQGKCPVDEETKKLWLREHGNEAHPGATAPGNQLECSANPQDNDKTPEYHTTVDLSQSREVSTIPRTNSDRNWIYPSEKQFYEAMMKKNWDPNSDDMKVVVPLHNSINERVWNYIKSWEDKQGGEACGGIKLTNFKGDSKKLTPRAWFRSRILHLAKPFDRHDWQIDRCGKTVDYVIDFYSTDLNDANSQQQPLIYLDVRPKLNSFEGFRLRFWKSLGF'

In [11]:
#testes
# results = {}
# for i in ID_PROT:
#     for field in fields:
#         result[field] = get_field_for_id(i, field)
#         results[i] = result
# print(results)

In [12]:
# #testes
# print(result_dict)
# print(get_list_uniprot(ID_PROT, result_dict))

In [15]:
#FINAL [idgenebank, protein_id, id do uniprot]  -  NECESSÁRIO
def join_ids_CDS(dic, uniprotID):
    join_list_all = []
    for key, value in dic.items():
        join_list = []
        join_list.append(key)
        join_list.append(value)
        join_list_all.append(join_list) 
    for index, values in enumerate(uniprotID):
        join_list_all[index].append(uniprotID[index][0])
    return join_list_all
join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))

[['Z17674.1', 'N/A_CDS', 'N/A'],
 ['V00883.1', 'CAA24252.1', 'P02099'],
 ['X67017.1', 'CAA47407.1', 'Q00873'],
 ['X61243.1', 'N/A_CDS', 'N/A']]

In [89]:
# print(join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))) #[idgenebank, protein_id, id do uniprot]
# print(get_CDS_info(Ids)) # [id_protein, location, product, translation]


[['X03907.1', 'CAA27540.1', 'P0CJ48'], ['L20655.1', 'AAA47844.1', 'Q88364']]
[["['CAA27540.1']", '[145 : 949]', "['chlorophyll a/b binding protein (LHCP AB 65)']", "['MAASTMALSSPAFAGKAVNLSPAASEVLGSGRVTMRKTVAKPKGPSGSPWYGSDRVKYLGPFSGESPSYLTGEFPGDYGWDTAGLSADPETFARNRELEVIHSRWAMLGALGCVFPELLARNGVKFGEAVWFKAGSQIFSDGGLDYLGNPSLVHAQSILAIWATQVILMGAVEGYRVAGNGPLGEAEDLLYPGGSFDPLGLATDPEAFAELKVKELKNGRLAMFSMFGFFVQAIVTGKGPIENLADHLADPVNNNAWAFATNFVPGK']"], ["['AAA47844.1']", '[<0 : >92]', "['gag protein']", "['LISEAETRGITGYNPLAGPLRVQANNPQQQG']"]]


In [20]:
#final [idgenebank, protein_id, id_uniprot,location, product, translation] (NÃO SEI SE ESTÁ A FUNCAR DIREITO - assumir que não)
def join_lists(list1, list2):
    result = []
    for item1 in list1:
        print(item1)
        for item2 in list2:
            print(item2)
            string_item2 = str(item2[0])
            string_item2 = string_item2.replace("[","").replace("]","").replace("'","")
            if item1[1] == string_item2 or item1[1] == string_item2:
                result.append(item1 + item2[1:])
    return result
join_lists(join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict)), get_CDS_info(Ids))


['X65482.1', 'N/A_CDS', 'N/A']
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['A00829.1', 'N/A_CDS', 'N/A']
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['M97702.1', 'N/A_CDS', 'N/A']
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['N/A_CDS

[['X65482.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['X65482.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['X65482.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['X65482.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['X65482.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['X65482.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['X65482.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['X65482.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['A00829.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['A00829.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['A00829.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['A00829.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['A00829.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['A00829.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['A00829.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['A00829.1', 'N/A_CDS', 'N/A', ['N/A'], ['N/A'], ['N/A']],
 ['M97702.1', 'N/A_CDS', 'N/A', ['N/A'],

In [108]:
# for i in join_lists(join_ids_CDS(result_dict,get_list_uniprot(ID_PROT)), get_CDS_info(Ids)):
#     print(i[4])

['chlorophyll a/b binding protein (LHCP AB 65)']
['gag protein']


### Começar povoar

In [50]:
# TESTES para povoar CDS
# print(result_dict)
# print(Ids)
# print(get_CDS_location(Ids)[0])
#key: id_genebank values: id_Uniprot
for i in get_CDS_info(Ids):
    print(i)

['N/A_CDS', ['N/A'], ['N/A'], ['N/A']]
['V00883.1', ['[223 : 316]', '[440 : 662]', '[1479 : 1608]'], ["['beta-3 globin']"], ["['MVHFTAEEKAAITSTWKLVDVEDAGAEALGRLLVVYPWTQRFFDSFGNLSSSSAIMGNPKVKAHGKKVLTAFGDAVKNVDDLKNTFAHLSELHCDRLHVDPENFKLLGNVLVIVLAKYFGKEFTPQVQSAWQKLVAGVATALAHKYH']"]]


In [38]:
#para povoar CDS

# get_CDS_info(Ids)[0][1]

['N/A']

In [26]:
#Povoação "CDS"

import mysql.connector as SQLC
import mysql.connector


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(Ids):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO CDS (ID_CDS, Product, Translation, Location, ID_genebank, ID_Uniprot) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val=(get_CDS_info(Ids)[index][0], get_CDS_info(Ids)[index][2], get_CDS_info(Ids)[index][3],get_CDS_info(Ids)[index][1],join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))[index][0],join_ids_CDS(result_dict,get_list_uniprot(ID_PROT, result_dict))[index][2])
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

MySQLInterfaceError: Python type list cannot be converted

In [367]:
#mini testes
# print(get_list_uniprot(ID_PROT)) #list of lists (ID_Uniprot, Function, Subcelular location, Protein seq, length_aa)
print(get_list_uniprot(ID_PROT)[0][0])
for index, value in enumerate(get_list_uniprot(ID_PROT)):
    print(str(get_list_uniprot(ID_PROT)[index][5]))

P02099
https://www.uniprot.org/uniprotkb/P02099/entry
https://www.uniprot.org/uniprotkb/Q00873/entry


In [17]:
#Povoação "Uniprot" - FEITO E FUNCIONA
# print(uniprot_final_list) #(ID_Uniprot, Subcelular location, Function, Protein seq, length_aa)

import mysql.connector as SQLC
import mysql.connector


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
#     sql = "ALTER TABLE Uniprot AUTO_INCREMENT = 0"
#     Cursor.execute(sql)
    for index, value in enumerate(get_list_uniprot(ID_PROT,result_dict)):
        #print(value)
        sql= "INSERT INTO Uniprot (ID_Uniprot, Subcelular_Location, Function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)"
        val=(get_list_uniprot(ID_PROT,result_dict)[index][0],get_list_uniprot(ID_PROT,result_dict)[index][1], get_list_uniprot(ID_PROT,result_dict)[index][2], get_list_uniprot(ID_PROT,result_dict)[index][3], get_list_uniprot(ID_PROT,result_dict)[index][4], get_list_uniprot(ID_PROT,result_dict)[index][5])
        Cursor.execute(sql,val)
        
        # SE UNIPROT_ID FOR AI
#     ID_AI=[]
#     sql= "Select ID_AI FROM Uniprot"
#     Cursor.execute(sql)
#     for row in Cursor:
#         ID_AI.append(str(row))
#     div_= ', '.join(ID_AI)
#     h_= div_.replace("(",'')
#     hh_= h_.replace(",)",'')
#     SEARCH_ID_AI= hh_.split(', ')
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

Erro na escrita na base de dados: 1062 (23000): Duplicate entry 'N/A' for key 'PRIMARY'


In [136]:
# print(SEARCH_ID_AI)

['7', '8']


In [130]:
# ORIGINAL para buscar os ids da tabela Uniprot, que sao AI
ID_AI=[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    sql= "Select ID_AI FROM Uniprot"
    Cursor.execute(sql)
    for row in Cursor:
        #print(row)
        ID_AI.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div_= ', '.join(ID_AI)
h_= div_.replace("(",'')
hh_= h_.replace(",)",'')
SEARCH_ID_AI= hh_.split(', ')
print(SEARCH_ID_AI)

['(1,)', '(2,)']
['1', '2']


In [ ]:
import pandas as pd 

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)

####### end of connection ####
tabela = pd.read_sql("Select * FROM Gene", DataBase)
DataBase.close()
print(tabela)